In [45]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
!pip install line-bot-sdk flask flask-ngrok


In [47]:
from flask import Flask, request, abort,jsonify
import json
from flask_ngrok import run_with_ngrok
from linebot import (
    LineBotApi,WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)

In [48]:
app = Flask(__name__,static_url_path = "/material", static_folder = "./material/")
run_with_ngrok(app)

line_bot_api = LineBotApi("9K2lbiSmQIY+xnkbt0MqxyD2SLrHvLY8nTb9Au76oyM+fG7ua4k6khpTN1TMM2aHNpQPDCAgSTgeT/hKZyS7SycCqx+OJaKWezw/tGlIk8P2rOtNIfgR9QEwmPokzcVmn4U1UW802Gs76jLK0cWShAdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("4257df85eafe8d3c8de0ca1cd8db08a9")

In [49]:
@app.route("/", methods=["POST"])
def callback():
  signature = request.headers['X-Line-Signature']
  body = request.get_data(as_text=True)
  print(body)
  f = open("/content/drive/MyDrive/ai-event.log","a")
  f.write(body)
  f.close
  try:
    handler.handle(body, signature)
  except InvalidSignatureError:
    abort(400)
  return "OK"


In [50]:
#引入所需要的消息與模板消息

from linebot.models import (
    MessageEvent, TemplateSendMessage, PostbackEvent
)
#引入按鍵模板
from linebot.models.template import (
    ButtonsTemplate
)

'''
alt_text: Line簡覽視窗所出現的說明文字
template: 所使用的模板
ButtonsTemplate: 按鍵模板
    thumbnail_image_url: 展示圖片
    title: 標題
    text: 說明文字
    actions: 模板行為所使用的行為
    data: 觸發postback後用戶回傳值，可以對其做商業邏輯處理

'''
buttons_template_message = TemplateSendMessage(
    alt_text = "AAA",
    template = ButtonsTemplate(
        #thumbnail_image_url = "https://www.executivecentre.com/wp-content/uploads/sites/34/2020/11/03015830/img-ciy_pass.jpg",
        title = "請幫幫我",
        text = "點擊下方按鈕求我幫你",
        actions = [
                  {
                      "type":"postback",
                      "label":"找找其他方法",
                      "text":"救救我啊我救我",
                      "data":"1234567890"
                  },
                  {
                      "type":"postback",
                      "label":"給我攻略",
                      "text":"我要callout",
                      "data":"5566"

                  }
                ],
    )
)

In [51]:
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)
# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
    "@more" : buttons_template_message,
}

In [52]:
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):

  if (event.message.text.find("@")!= -1):
    line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
    )
  else:
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="你輸入什麼呀！？如果不知道就輸入@more吧！")
    )


In [53]:
@handler.add(PostbackEvent)
def handle_post_message(event):
  user_profile = line_bot_api.get_profile(event.source.user_id)
  if (event.postback.data.find("1234567890")==0):
    with open("user_profile_business.txt","a") as myfile:
      myfile.write(json.dumps(vars(user_profile),sort_keys=True))
      myfile.write("\n")
      line_bot_api.reply_message(
          event.reply_token,
          TextMessage(
              text="你等等，我應該會叫人來幫你"
          )
      )
  elif (event.postback.data.find("5566")== 0):
      with open("user_profile_tutorial.txt","a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write("\n")
        line_bot_api.reply_message(
            event.reply_token,
            TextMessage(
                text="請稍等，給我一百我就叫專家跟你談"
            )
        )
  else:
      pass

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c4ea2aaf0468.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"destination":"U84891643b3f7d23f39ca3989530192c5","events":[{"type":"message","message":{"type":"text","id":"14108334684900","text":"@more"},"timestamp":1621887558914,"source":{"type":"user","userId":"U2f25cfa41ca384946f24585d45019074"},"replyToken":"f368debd51c6436c9f9b8638bd262777","mode":"active"}]}


127.0.0.1 - - [24/May/2021 20:19:20] "POST / HTTP/1.1" 200 -


{"destination":"U84891643b3f7d23f39ca3989530192c5","events":[{"type":"postback","postback":{"data":"1234567890"},"timestamp":1621887579699,"source":{"type":"user","userId":"U2f25cfa41ca384946f24585d45019074"},"replyToken":"47c5d922644b4446947f7b9f83269e71","mode":"active"},{"type":"message","message":{"type":"text","id":"14108335418052","text":"救救我啊我救我"},"timestamp":1621887579700,"source":{"type":"user","userId":"U2f25cfa41ca384946f24585d45019074"},"replyToken":"6a1e6a45cd374b16a3dcc7b26c7c16e3","mode":"active"}]}


127.0.0.1 - - [24/May/2021 20:19:41] "POST / HTTP/1.1" 200 -


{"destination":"U84891643b3f7d23f39ca3989530192c5","events":[{"type":"postback","postback":{"data":"5566"},"timestamp":1621887586373,"source":{"type":"user","userId":"U2f25cfa41ca384946f24585d45019074"},"replyToken":"0d6808b94e0c4f3a9ef9daee99468f37","mode":"active"},{"type":"message","message":{"type":"text","id":"14108335648564","text":"我要callout"},"timestamp":1621887586373,"source":{"type":"user","userId":"U2f25cfa41ca384946f24585d45019074"},"replyToken":"4856fc18be424e478fe19bd3ef942b26","mode":"active"}]}


127.0.0.1 - - [24/May/2021 20:19:48] "POST / HTTP/1.1" 200 -
